__Probing Language Models__

This notebook serves as a start for your NLP2 assignment on probing Language Models. This notebook will become part of the contents that you will submit at the end, so make sure to keep your code (somewhat) clean :-)

__note__: This is only the second time anyone is doing this assignment. That's exciting! But it might well be the case that certain aspects are too unclear. Do not hesitate at all to reach to me once you get stuck, I'd be grateful to help you out.

__note 2__: This assignment is not dependent on big fancy GPUs. I run all this stuff on my own 3 year old CPU, without any Colab hassle. So it's up to you to decide how you want to run it.

# Models

For the Transformer models you are advised to make use of the `transformers` library of Huggingface: https://github.com/huggingface/transformers
Their library is well documented, and they provide great tools to easily load in pre-trained models.

In [12]:
#
## Your code for initializing the transformer model(s)
#
# Note that most transformer models use their own `tokenizer`, that should be loaded in as well.
#
from transformers import pipeline, GPT2Model, GPT2TokenizerFast



gpt2 = GPT2Model.from_pretrained("distilgpt2")
tokenizer = GPT2TokenizerFast.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
gpt2.resize_token_embeddings(len(tokenizer))
gpt2.config.pad_token_id = gpt2.config.eos_token_id
tokenizer.is_split_into_words=True
# tokenizer.add_prefix_space=True

# Note that some models don't return the hidden states by default.
# This can be configured by passing `output_hidden_states=True` to the `from_pretrained` method.

Some weights of GPT2Model were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
#
## Your code for initializing the rnn model(s)
#
# The Gulordava LSTM model can be found here: 
# https://drive.google.com/file/d/19Lp3AM4NEPycp_IBgoHfLc_V456pmUom/view?usp=sharing
#
# N.B: I have altered the RNNModel code to only output the hidden states that you are interested in.
# If you want to do more experiments with this model you could have a look at the original code here:
# https://github.com/facebookresearch/colorlessgreenRNNs/blob/master/src/language_models/model.py
#
from collections import defaultdict
from lstm.model import RNNModel
import torch


model_location = 'lstm/state_dict.pt'  # <- point this to the location of the Gulordava .pt file
lstm = RNNModel('LSTM', 50001, 650, 650, 2)
lstm.load_state_dict(torch.load(model_location))


# This LSTM does not use a Tokenizer like the Transformers, but a Vocab dictionary that maps a token to an id.
with open('lstm/vocab.txt', encoding='utf-8') as f:
        w2i = {w.strip(): i for i, w in enumerate(f)}

vocab = defaultdict(lambda: w2i["<unk>"])
vocab.update(w2i)

It is a good idea that before you move on, you try to feed some text to your LMs; and check if everything works accordingly. 

# Data

For this assignment you will train your probes on __treebank__ corpora. A treebank is a corpus that has been *parsed*, and stored in a representation that allows the parse tree to be recovered. Next to a parse tree, treebanks also often contain information about part-of-speech tags, which is exactly what we are after now.

The treebank you will use for now is part of the Universal Dependencies project. I provide a sample of this treebank as well, so you can test your setup on that before moving on to larger amounts of data.

Make sure you accustom yourself to the format that is created by the `conllu` library that parses the treebank files before moving on. For example, make sure you understand how you can access the pos tag of a token, or how to cope with the tree structure that is formed using the `to_tree()` functionality.

In [14]:
# READ DATA
from typing import List
from conllu import parse_incr, TokenList


# If stuff like `: str` and `-> ..` seems scary, fear not! 
# These are type hints that help you to understand what kind of argument and output is expected.
def parse_corpus(filename: str) -> List[TokenList]:
    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses

Parse_corpus returns a list of TokenLists. A TokenList represents a single sentence and is a list of dicts. 
There is one dict for every token in the sentence, containing:

    ID: Word index, integer starting at 1 for each new sentence; may be a range for multiword tokens; may be a decimal number for empty nodes (decimal numbers can be lower than 1 but must be greater than 0).
    FORM: Word form or punctuation symbol.
    LEMMA: Lemma or stem of word form.
    UPOS: Universal part-of-speech tag.
    XPOS: Language-specific part-of-speech tag; underscore if not available.
    FEATS: List of morphological features from the universal feature inventory or from a defined language-specific extension; underscore if not available.
    HEAD: Head of the current word, which is either a value of ID or zero (0).
    DEPREL: Universal dependency relation to the HEAD (root iff HEAD = 0) or a defined language-specific subtype of one.
    DEPS: Enhanced dependency graph in the form of a list of head-deprel pairs.
    MISC: Any other annotation.



# Generating Representations

We now have our data all set, our models are running and we are good to go!

The next step is now to create the model representations for the sentences in our corpora. Once we have generated these representations we can store them, and train additional diagnostic (/probing) classifiers on top of the representations.

There are a few things you should keep in mind here. Read these carefully, as these tips will save you a lot of time in your implementation.

1. Transformer models make use of Byte-Pair Encodings (BPE), that chunk up a piece of next in subword pieces. For example, a word such as "largely" could be chunked up into "large" and "ly". We are interested in probing linguistic information on the __word__-level. Therefore, we will follow the suggestion of Hewitt et al. (2019a, footnote 4), and create the representation of a word by averaging over the representations of its subwords. So the representation of "largely" becomes the average of that of "large" and "ly".


2. Subword chunks never overlap multiple tokens. In other words, say we have a phrase like "None of the", then the tokenizer might chunk that into "No"+"ne"+" of"+" the", but __not__ into "No"+"ne o"+"f the", as those chunks overlap multiple tokens. This is great for our setup! Otherwise it would have been quite challenging to distribute the representation of a subword over the 2 tokens it belongs to.


3. **Important**: If you closely examine the provided treebank, you will notice that some tokens are split up into multiple pieces, that each have their own POS-tag. For example, in the first sentence the word "Al-Zaman" is split into "Al", "-", and "Zaman". In such cases, the conllu `TokenList` format will add the following attribute: `('misc', OrderedDict([('SpaceAfter', 'No')]))` to these tokens. Your model's tokenizer does not need to adhere to the same tokenization. E.g., "Al-Zaman" could be split into "Al-"+"Za"+"man", making it hard to match the representations with their correct pos-tag. Therefore I recommend you to not tokenize your entire sentence at once, but to do this based on the chunking of the treebank. <br /><br />
Make sure to still incoporate the spaces in a sentence though, as these are part of the BPE of the tokenizer. That is, the tokenizer uses a different token id for `"man"`, than it does for `" man"`: the former could be part of `" woman"`=`" wo`"+`"man"`, whereas the latter would be the used in case *man* occurs at the start of a word. The tokenizer for GPT-2 adds spaces at the start of a token (represented as a `Ġ` symbol). This means that you should keep track whether the previous token had the `SpaceAfter` attribute set to `'No'`: in case it did not, you should manually prepend a `" "` ahead of the token.


4. The LSTM LM does not have the issues related to subwords, but is far more restricted in its vocabulary. Make sure you keep the above points in mind though, when creating the LSTM representations. You might want to write separate functions for the LSTM, but that is up to you.


5. **N.B.**: Make sure that when you run a sentence through your model, you do so within a `with torch.no_grad():` block, and that you have run `model.eval()` beforehand as well (to disable dropout).


6. **N.B.**: Make sure to use a token's ``["form"]`` attribute, and not the ``["lemma"]``, as the latter will stem any relevant morphological information from the token. We don't want this, because we want to feed well-formed, grammatical sentences to our model.


I would like to stress that if you feel hindered in any way by the simple code structure that is presented here, you are free to modify it :-) Just make sure it is clear to an outsider what you're doing, some helpful comments never hurt.

In [15]:
# FETCH SENTENCE REPRESENTATIONS
from torch import Tensor
import pickle


# Should return a tensor of shape (num_tokens_in_corpus, representation_size)
# Make sure you correctly average the subword representations that belong to 1 token!

def fetch_sen_reps(ud_parses: List[TokenList], model, tokenizer, concat=False) -> Tensor:
    # output needs to be of size (n_sentences, max_seq_len, emb_dim)
    # lstm emb_dim = 650, gpt2 emb_dim = 768
    max_len = max([len(tokenlist) for tokenlist in ud_parses])

    if isinstance(tokenizer, defaultdict):
        output = torch.zeros((len(ud_parses), max_len, 650))

        for i in range(len(ud_parses)):
            tokenlist = ud_parses[i]
            tokens = [tokenizer[token['form']] for token in tokenlist]
            encoded_input = torch.as_tensor(tokens).reshape(1, -1)
            model.eval()
            with torch.no_grad():
                hidden = model.init_hidden(encoded_input.shape[0])
                result = model(encoded_input, hidden).squeeze()
            output[i, :len(tokenlist), :] = result

    else:
        output = torch.zeros((len(ud_parses), max_len, 768))
        for i  in range(len(ud_parses)):
            tokenlist = ud_parses[i]
            space = False
            ids = []
            mask = []

            for j in range(len(tokenlist)):
                token = tokenlist[j]
                form = " " + token['form'] if space else token['form']
                space = True if not token['misc'] else False
                tokenized = tokenizer(form)
                seq_length = len(tokenized['input_ids'])
                result = 0
                for k in range(seq_length):
                    ids.append(tokenized['input_ids'][k])
                    mask.append(tokenized['attention_mask'][k])
                    encoded_input = {'input_ids': torch.as_tensor(ids), 'attention_mask': torch.as_tensor(mask)}
                    model.eval()
                    with torch.no_grad():
                        result += model(**encoded_input).last_hidden_state[-1, :]

                result /= seq_length
                output[i, j, :] = result
            
    output = output.squeeze()
    return output

In [16]:
def test_sen_reps():
    corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')[:1]
    own_distilgpt2_emb1 = fetch_sen_reps(corpus, gpt2, tokenizer)

test_sen_reps()

To validate your activation extraction procedure I have set up the following assertion function as a sanity check. It compares your representation against a pickled version of mine. 

For this I used `distilgpt2`.

In [17]:
def error_msg(model_name, gold_embs, embs, i2w):
    with open(f'{model_name}_tokens1.pickle', 'rb') as f:
        sen_tokens = pickle.load(f)
        
    diff = torch.abs(embs - gold_embs)
    max_diff = torch.max(diff)
    avg_diff = torch.mean(diff)
    
    print(f"{model_name} embeddings don't match!")
    print(f"Max diff.: {max_diff:.4f}\nMean diff. {avg_diff:.4f}")

    print("\nCheck if your tokenization matches with the original tokenization:")
    for idx in sen_tokens.squeeze():
        if isinstance(i2w, list):
            token = i2w[idx]
        else:
            token = i2w.convert_ids_to_tokens(idx.item())
        print(f"{idx:<6} {token}")


def assert_sen_reps(gpt2, tokenizer, lstm, vocab):
    with open('distilgpt2_emb1.pickle', 'rb') as f:
        distilgpt2_emb1 = pickle.load(f)
        
    with open('lstm_emb1.pickle', 'rb') as f:
        lstm_emb1 = pickle.load(f)
    
    corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')[:1]
    
    own_distilgpt2_emb1 = fetch_sen_reps(corpus, gpt2, tokenizer)
    own_lstm_emb1 = fetch_sen_reps(corpus, lstm, vocab)
    assert distilgpt2_emb1.shape == own_distilgpt2_emb1.shape, \
        f"Distilgpt2 shape mismatch: {distilgpt2_emb1.shape} (gold) vs. {own_distilgpt2_emb1.shape} (yours)"
    assert lstm_emb1.shape == own_lstm_emb1.shape, \
        f"LSTM shape mismatch: {lstm_emb1.shape} (gold) vs. {own_lstm_emb1.shape} (yours)"

    if not torch.allclose(distilgpt2_emb1, own_distilgpt2_emb1, rtol=1e-3, atol=1e-3):
        error_msg("distilgpt2", distilgpt2_emb1, own_distilgpt2_emb1, tokenizer)
    if not torch.allclose(lstm_emb1, own_lstm_emb1, rtol=1e-3, atol=1e-3):
        error_msg("lstm", lstm_emb1, own_lstm_emb1, list(vocab.keys()))

assert_sen_reps(gpt2, tokenizer, lstm, vocab)

Next, we should define a function that extracts the corresponding POS labels for each activation, which we do based on the **``"upostag"``** attribute of a token (so not the ``xpostag`` attribute). These labels will be transformed to a tensor containing the label index for each item.

Upos-tags are assigned in the following order:

1. ADJ: adjective
2. ADP: adposition
3. ADV: adverb
4. AUX: auxiliary
5. CCONJ: coordinating conjunction
6. DET: determiner
7. INTJ: interjection
8. NOUN: noun
9. NUM: numeral
10. PART: particle
11. PRON: pronoun
12. PROPN: proper noun
13. PUNCT: punctuation
14. SCONJ: subordinating conjunction
15. SYM: symbol
16. VERB: verb
17. X: other

In [18]:
# FETCH POS LABELS

# Should return a tensor of shape (num_tokens_in_corpus,)
# Make sure that when fetching these pos tags for your train/dev/test corpora you share the label vocabulary.
def fetch_pos_tags(ud_parses: List[TokenList], pos_vocab=None) -> Tensor:
    pos_vocab = {'ADJ': 1, 'ADP': 2, 'ADV': 3, 'AUX': 4, 'CCONJ': 5, 'DET': 6, 'INTJ': 7, 'NOUN': 8, 'NUM': 9, 'PART': 10, 'PRON': 11, 'PROPN': 12, 'PUNCT': 13, 'SCONJ': 14, 'SYM': 15, 'VERB': 16, 'X': 17}

    pos_list = [pos_vocab[token['upostag']] for tokenlist in ud_parses for token in tokenlist]
    return torch.as_tensor(pos_list).reshape(-1,), pos_vocab


In [20]:
import os

# Function that combines the previous functions, and creates 2 tensors for a .conllu file: 
# 1 containing the token representations, and 1 containing the (tokenized) pos_tags.

def create_data(filename: str, lm, w2i, pos_vocab=None):
    ud_parses = parse_corpus(filename)
    
    sen_reps = fetch_sen_reps(ud_parses, lm, w2i)
    pos_tags, pos_vocab = fetch_pos_tags(ud_parses, pos_vocab=pos_vocab)
    
    return sen_reps, pos_tags, pos_vocab


lm = gpt2  # or `lstm`
w2i = tokenizer  # or `vocab`
use_sample = True

train_x, train_y, train_vocab = create_data(
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-train.conllu'),
    lm, 
    w2i
)

dev_x, dev_y, _ = create_data(
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-dev.conllu'),
    lm, 
    w2i,
    pos_vocab=train_vocab
)

test_x, test_y, _ = create_data(
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-test.conllu'),
    lm,
    w2i,
    pos_vocab=train_vocab
)

# Diagnostic Classification

We now have our models, our data, _and_ our representations all set! Hurray, well done. We can finally move onto the cool stuff, i.e. training the diagnostic classifiers (DCs).

DCs are simple in their complexity on purpose. To read more about why this is the case you could already have a look at the "Designing and Interpreting Probes with Control Tasks" by Hewitt and Liang (esp. Sec. 3.2).

A simple linear classifier will suffice for now, don't bother with adding fancy non-linearities to it.

I am personally a fan of the `skorch` library, that provides `sklearn`-like functionalities for training `torch` models, but you are free to train your dc using whatever method you prefer.

As this is an Artificial Intelligence master and you have all done ML1 + DL, I expect you to use your train/dev/test splits correctly ;-)

In [21]:
# DIAGNOSTIC CLASSIFIER



# Trees

For our gold labels, we need to recover the node distances from our parse tree. For this we will use the functionality provided by `ete3`, that allows us to compute that directly. I have provided code that transforms a `TokenTree` to a `Tree` in `ete3` format.

In [64]:
# In case you want to transform your conllu tree to an nltk.Tree, for better visualisation

def rec_tokentree_to_nltk(tokentree):
    token = tokentree.token["form"]
    tree_str = f"({token} {' '.join(rec_tokentree_to_nltk(t) for t in tokentree.children)})"

    return tree_str


def tokentree_to_nltk(tokentree):
    from nltk import Tree as NLTKTree

    tree_str = rec_tokentree_to_nltk(tokentree)

    return NLTKTree.fromstring(tree_str)

In [65]:
# !pip install ete3
from ete3 import Tree as EteTree


class FancyTree(EteTree):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, format=1, **kwargs)
        
    def __str__(self):
        return self.get_ascii(show_internal=True)
    
    def __repr__(self):
        return str(self)


def rec_tokentree_to_ete(tokentree):
    idx = str(tokentree.token["id"])
    children = tokentree.children
    if children:
        return f"({','.join(rec_tokentree_to_ete(t) for t in children)}){idx}"
    else:
        return idx
    
def tokentree_to_ete(tokentree):
    newick_str = rec_tokentree_to_ete(tokentree)

    return FancyTree(f"{newick_str};")

In [58]:
# Let's check if it works!
# We can read in a corpus using the code that was already provided, and convert it to an ete3 Tree.

def parse_corpus(filename):
    from conllu import parse_incr

    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses

def test_ete():
    corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')
    item = corpus[0]
    tokentree = item.to_tree()
    ete3_tree = tokentree_to_ete(tokentree)
    print(ete3_tree)

test_ete()

NameError: name 'tokentree_to_ete' is not defined

As you can see we label a token by its token id (converted to a string). Based on these id's we are going to retrieve the node distances.

To create the true distances of a parse tree in our treebank, we are going to use the `.get_distance` method that is provided by `ete3`: http://etetoolkit.org/docs/latest/tutorial/tutorial_trees.html#working-with-branch-distances

We will store all these distances in a `torch.Tensor`.

Please fill in the gap in the following method. I recommend you to have a good look at Hewitt's blog post  about these node distances.

From https://nlp.stanford.edu/~johnhew/structural-probe.html#fnref:1



In [63]:
def create_gold_distances(corpus):
    max_len = max([len(tokenlist) for tokenlist in corpus])
    all_distances = torch.zeros(len(corpus), max_len, max_len)

    for i in range(len((corpus))):
        item = corpus[i]
        tokentree = item.to_tree()
        ete_tree = tokentree_to_ete(tokentree)
        nodes = ete_tree.search_nodes()
        sen_len = len(nodes)
        distances = torch.zeros((sen_len, sen_len))

        # Your code for computing all the distances comes here.
        for j in range(sen_len):
            for k in range(sen_len):
                 all_distances[i, j, k] = nodes[j].get_distance(nodes[k])

    return all_distances

def test_gold_distances():
    corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')
    print(create_gold_distances(corpus[2:4]))

In [64]:
test_gold_distances()

tensor([[[ 0.,  1.,  1.,  1.,  2.,  2.,  3.,  3.,  3.,  3.,  3.,  3.,  4.,  4.,
           4.,  5.,  6.],
         [ 1.,  0.,  2.,  2.,  3.,  3.,  4.,  4.,  4.,  4.,  4.,  4.,  5.,  5.,
           5.,  6.,  7.],
         [ 1.,  2.,  0.,  2.,  1.,  1.,  2.,  2.,  2.,  2.,  2.,  2.,  3.,  3.,
           3.,  4.,  5.],
         [ 1.,  2.,  2.,  0.,  3.,  3.,  4.,  4.,  4.,  4.,  4.,  4.,  5.,  5.,
           5.,  6.,  7.],
         [ 2.,  3.,  1.,  3.,  0.,  2.,  1.,  3.,  3.,  3.,  3.,  3.,  4.,  4.,
           4.,  5.,  6.],
         [ 2.,  3.,  1.,  3.,  2.,  0.,  3.,  1.,  1.,  1.,  1.,  1.,  2.,  2.,
           2.,  3.,  4.],
         [ 3.,  4.,  2.,  4.,  1.,  3.,  0.,  4.,  4.,  4.,  4.,  4.,  5.,  5.,
           5.,  6.,  7.],
         [ 3.,  4.,  2.,  4.,  3.,  1.,  4.,  0.,  2.,  2.,  2.,  2.,  3.,  3.,
           3.,  4.,  5.],
         [ 3.,  4.,  2.,  4.,  3.,  1.,  4.,  2.,  0.,  2.,  2.,  2.,  3.,  3.,
           3.,  4.,  5.],
         [ 3.,  4.,  2.,  4.,  3.,  1.,  4.,  

The next step is now to do the previous step the other way around. After all, we are mainly interested in predicting the node distances of a sentence, in order to recreate the corresponding parse tree.

Hewitt et al. reconstruct a parse tree based on a _minimum spanning tree_ (MST, https://en.wikipedia.org/wiki/Minimum_spanning_tree). Fortunately for us, we can simply import a method from `scipy` that retrieves this MST.

In [65]:
from scipy.sparse.csgraph import minimum_spanning_tree
import torch


def create_mst(distances):
    distances = torch.triu(distances).detach().numpy()
    
    mst = minimum_spanning_tree(distances).toarray()
    mst[mst>0] = 1.
    
    return mst

Let's have a look at what this looks like, by looking at a relatively short sentence in the sample corpus.

If your addition to the `create_gold_distances` method has been correct, you should be able to run the following snippet. This then shows you the original parse tree, the distances between the nodes, and the MST that is retrieved from these distances. Can you spot the edges in the MST matrix that correspond to the edges in the parse tree?

In [77]:
corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')
item = corpus[5]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
# print(ete3_tree, '\n')

gold_distance = create_gold_distances(corpus[2:3])[0]
print(gold_distance, '\n')

mst = create_mst(gold_distance)
print(mst)

tensor([[ 0.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1.,  0.,  2.,  ..., -1., -1., -1.],
        [ 1.,  2.,  0.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]]) 

[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0. -1.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


Now that we are able to map edge distances back to parse trees, we can create code for our quantitative evaluation. For this we will use the Undirected Unlabeled Attachment Score (UUAS), which is expressed as:

$$\frac{\text{number of predicted edges that are an edge in the gold parse tree}}{\text{number of edges in the gold parse tree}}$$

To do this, we will need to obtain all the edges from our MST matrix. Note that, since we are using undirected trees, that an edge can be expressed in 2 ways: an edge between node $i$ and node $j$ is denoted by both `mst[i,j] = 1`, or `mst[j,i] = 1`.

You will write code that computes the UUAS score for a matrix of predicted distances, and the corresponding gold distances. I recommend you to split this up into 2 methods: 1 that retrieves the edges that are present in an MST matrix, and one general method that computes the UUAS score.

In [ ]:
def edges(mst):
    edges = set()

    # Your code for retrieving the edges from the MST matrix
    mst = torch.tensor(mst)
    edges = torch.nonzero(mst).long()
    return edges


def calc_uuas(pred_edges, gold_distances):
    uuas = 0
    n_correct = 0'
    
    # Your code for computing the UUAS score
    for i in range(pred_edges.shape[0]):
        for edge in pred_edges[i]:
            print(pred_edges[i])
            edge = edge.long()
            if gold_distances[edge[0], edge[1]] == 1:
                n_correct += 1
        gold_distances = gold_distances.long()
        n_gold = torch.sum(torch.where(gold_distances == 1, gold_distances, 0))/2
        uuas += n_correct/n_gold

    uuas /= pred_edges.shape[1]
    return uuas


def test_calc_uuas():
    gold_distances = create_gold_distances(corpus)
    for gold_distance in gold_distances:
        mst = create_mst(gold_distance)
        pred_edges = edges(mst)
        uuas = calc_uuas(pred_edges, gold_distance)
        if uuas != 1:
            print(uuas)
    print('Done')



test_calc_uuas()

# Structural Probes

We now have everything in place to start doing the actual exciting stuff: training our structural probe!
    
To make life easier for you, we will simply take the `torch` code for this probe from John Hewitt's repository. This allows you to focus on the training regime from now on.

I have provided a rough outline for the training regime that you can use. Note that the hyper parameters that I provide here only serve as an indication, but should be (briefly) explored by yourself.

As can be seen in Hewitt's code above, there exists functionality in the probe to deal with batched input. It is up to you to use that: a (less efficient) method can still incorporate batches by doing multiple forward passes for a batch and computing the backward pass only once for the summed losses of all these forward passes. (_I know, this is not the way to go, but in the interest of time that is allowed ;-), the purpose of the assignment is writing a good paper after all_).

In [108]:
def evaluate_probe(loss_function, probe, data, labels, sen_lens):
    predictions = probe.forward(data)
    loss_score, total_sents = loss_function(predictions, labels, sen_lens)
    # uuas_score = calc_uuas(predictions, labels)
    uuas_score = 0
    return loss_score, uuas_score

In [109]:
from torch import optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np

def init_corpus(path, model, concat=False, cutoff=None):
    """ Initialises the data of a corpus.
    
    Parameters
    ----------
    path : str
        Path to corpus location
    concat : bool, optional
        Optional toggle to concatenate all the tensors
        returned by `fetch_sen_reps`.
    cutoff : int, optional
        Optional integer to "cutoff" the data in the corpus.
        This allows only a subset to be used, alleviating 
        memory usage.

        N.B. I have switched fetch_sen_reps to batch_sen_reps and the output so first embs 
        and then gold_distances is returned.
    """

    corpus = parse_corpus(path)[:cutoff]
    embs = fetch_sen_reps(corpus, model, tokenizer, concat=concat)    
    gold_distances = create_gold_distances(corpus)
    sen_lens = torch.tensor([len(tokenlist) for tokenlist in corpus])
    return embs, gold_distances, sen_lens



In [110]:
def test_init_corpus():
    cutoff = 2
    model = gpt2
    embs, gold_distances, sen_lens = init_corpus('data/sample/en_ewt-ud-train.conllu', model=model, cutoff=cutoff)
    print(type(gold_distances))
    print(type(gold_distances[0]))
    

test_init_corpus()


<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [122]:
def train(model, params=None, cutoff=None):

    # set parameters
    _params = {'epochs': 10, 'lr': 10e-4, 'batch_size': 24, 'rank': 64}
    for k, v in params.items():
        _params[k] = v

    epochs = _params['epochs']
    lr = _params['lr']
    batch_size = _params['batch_size']
    emb_dim = 768 if model == gpt2 else 650
    rank = _params['rank']

    #initalise the data
    train_x, train_y, train_lens = init_corpus('data/sample/en_ewt-ud-train.conllu', model=model, cutoff=cutoff)
    dev_x, dev_y, dev_lens = init_corpus('data/sample/en_ewt-ud-dev.conllu', model=model, cutoff=cutoff)
    test_x, test_y, test_lens = init_corpus('data/sample/en_ewt-ud-test.conllu',model=model, cutoff=cutoff)

    probe = StructuralProbe(emb_dim, rank)
    optimizer = optim.Adam(probe.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5,patience=1)
    loss_function =  L1DistanceLoss()

    #Set up logging
    writer = SummaryWriter()

    for epoch in range(epochs):
        print(epoch)
        for i in range(0, train_x.shape[0], batch_size):
            print(i)
            optimizer.zero_grad()

            # The forward pass
            batch_data = train_x[i:i+batch_size, :, :]
            batch_labels = train_y[i:i+batch_size]
            preds = probe.forward(batch_data)

            #Computing Training loss
            sentence_lengths = 0
            train_loss = loss_function.forward(preds, batch_labels, train_lens)[0]
            train_loss.backward()
            optimizer.step()

        #Log last training loss on the epoch
        writer.add_scalar('Loss/train', train_loss, epoch)
      
        #Compute dev loss for epoch
        dev_loss, dev_uuas = evaluate_probe(loss_function, probe, dev_x, dev_y, dev_lens)
        writer.add_scalar('Loss/dev', dev_loss, epoch)
        writer.add_scalar('UUAS/dev', dev_uuas, epoch)

        # Using a scheduler is up to you, and might require some hyper param fine-tuning
        scheduler.step(dev_loss)

    test_loss, test_uuas = evaluate_probe(loss_function, probe, test_x, test_y, test_lens)
    writer.add_scalar('Loss/test', test_loss, epochs-1)
    writer.add_scalar('UUAS/test', test_uuas, epochs-1)

    write.close()

In [123]:
import torch.nn as nn
import torch


class StructuralProbe(nn.Module):
    """ Computes squared L2 distance after projection by a matrix.
    For a batch of sentences, computes all n^2 pairs of distances
    for each sentence in the batch.
    """
    def __init__(self, model_dim, rank, device="cpu"):
        super().__init__()
        self.probe_rank = rank
        self.model_dim = model_dim
        
        self.proj = nn.Parameter(data = torch.zeros(self.model_dim, self.probe_rank))
        
        nn.init.uniform_(self.proj, -0.05, 0.05)
        self.to(device)

    def forward(self, batch):
        """ Computes all n^2 pairs of distances after projection
        for each sentence in a batch.
        Note that due to padding, some distances will be non-zero for pads.
        Computes (B(h_i-h_j))^T(B(h_i-h_j)) for all i,j
        Args:
          batch: a batch of word representations of the shape
            (batch_size, max_seq_len, representation_dim)
        Returns:
          A tensor of distances of shape (batch_size, max_seq_len, max_seq_len)
        """

        transformed = torch.matmul(batch, self.proj)
        batchlen, seqlen, rank = transformed.size()
        
        transformed = transformed.unsqueeze(2)
        transformed = transformed.expand(-1, -1, seqlen, -1)
        transposed = transformed.transpose(1,2)
        
        diffs = transformed - transposed
        
        squared_diffs = diffs.pow(2)
        squared_distances = torch.sum(squared_diffs, -1)

        return squared_distances

    
class L1DistanceLoss(nn.Module):
    """Custom L1 loss for distance matrices."""
    def __init__(self):
        super().__init__()

    def forward(self, predictions, label_batch, length_batch):
        """ Computes L1 loss on distance matrices.
        Ignores all entries where label_batch=-1
        Normalizes first within sentences (by dividing by the square of the sentence length)
        and then across the batch.
        Args:
          predictions: A pytorch batch of predicted distances
          label_batch: A pytorch batch of true distances
          length_batch: A pytorch batch of sentence lengths
        Returns:
          A tuple of:
            batch_loss: average loss in the batch
            total_sents: number of sentences in the batch
        """
        # Labels_1s will always be all 1's, as I do not include -1's in gold_distances,
        # since this breaks the MST calculations.

        label_batch
        labels_1s = (label_batch != -1).float()
        predictions_masked = predictions * labels_1s
        labels_masked = label_batch * labels_1s
        
        total_sents = torch.sum((length_batch != 0)).float()
        squared_lengths = length_batch.pow(2).float()

        if total_sents > 0:
            # loss_per_sent = torch.sum(torch.abs(predictions_masked - labels_masked), dim=(1,2))
            loss_per_sent = torch.sum(torch.abs(predictions - label_batch), dim=(1,2))
            normalized_loss_per_sent = loss_per_sent / squared_lengths
            batch_loss = torch.sum(normalized_loss_per_sent) / total_sents
        
        else:
            batch_loss = torch.tensor(0.0)
        
        return batch_loss, total_sents


tensor([1., 1., 1., 1., 0.])


In [124]:

#available models are gpt2 and lstm
#model = gpt2
#model = lstm
# default params: rank = 64, lr = 10e-4, batch_size = 24, epochs = 10
#emb_dim needs to be 768 for gpt2 and 650 for lstm
rank = 64
lr = 10e-4
batch_size = 2
epochs = 10
params = {'epochs': epochs, 'lr': lr, 'batch_size': batch_size, 'rank': rank}
train(gpt2, params, cutoff=100)

0
0


RuntimeError: The size of tensor a (2) must match the size of tensor b (90) at non-singleton dimension 0

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "c:\users\aukee\miniconda3\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\aukee\miniconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\aukee\miniconda3\Scripts\tensorboard.exe\__main__.py", line 4, in <module>
  File "c:\users\aukee\miniconda3\lib\site-packages\tensorboard\main.py", line 43, in <module>
    from tensorboard import default
  File "c:\users\aukee\miniconda3\lib\site-packages\tensorboard\default.py", line 36, in <module>
    from tensorboard.plugins.audio import audio_plugin
  File "c:\users\aukee\miniconda3\lib\site-packages\tensorboard\plugins\audio\audio_plugin.py", line 27, in <module>
    from tensorboard.data import provider
  File "c:\users\aukee\miniconda3\lib\site-packages\tensorboard\data\__init__.py", line 20, in <module>
    from tensorboard.data

Structural Control task

For every tree, make sure that:

all nodes are connected to either the first or a child of the first node



In [105]:
import numpy as np
import random
from ete3 import Tree as EteTree


def create_control_trees(corpus, depth=2):
    trees = [] * len(corpus)
    for i in range(len((corpus))):
            item = corpus[i]
            tokentree = item.to_tree()
            ete_tree = tokentree_to_ete(tokentree)
            print(ete_tree)
            levels = [[] for i in range(depth)]
            root = ete_tree.get_tree_root().detach()
            for node in ete_tree.iter_descendants('preorder'):
                level = random.choice(levels)
                level.append(node.detach())
             
            levels = [level for level in levels if len(level) != 0]

            for i in range(len(levels)):
                if i == 0:
                    for node in levels[0]:
                        root.add_child(node)
                else:
                    for node in levels[i]:
                        parent = random.choice(levels[i-1])
                        parent.add_child(node)
           
            trees.append(ete_tree)
    return trees




In [109]:
from ete3 import Tree
t = Tree("(A:1,(B:1,(E:1,D:1):0.5):0.5);" )

def test_control_tree():
    corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')[:2]
    tree = create_control_trees(corpus, depth=5)
    print(tree)

test_control_tree()




   /-2
  |
  |--3
  |
  |--4
  |
  |   /6 /-5
  |  |
  |  |   /-9
  |  |  |
  |  |  |--10
  |  |  |
  |  |  |--11
  |  |-8|
  |  |  |--12
  |-7|  |
  |  |  |--13
  |  |  |
  |  |   \15/-14
-1|  |
  |  |   /-16
  |  |  |
  |  |  |--17
  |  |  |
  |   \18   /-19
  |     |  |
  |     |  |--20
  |     |  |
  |     |  |-23/-22
  |      \21
  |        |--24
  |        |
  |        |   /-25
  |        |  |
  |         \28--26
  |           |
  |            \-27
  |
   \-29

   /-1
  |
  |   /-2
  |  |
  |-3|   /-4
  |  |  |
  |   \7|--5
  |     |
  |      \-6
  |
  |--8
  |
-10--9
  |
  |--11
  |
  |--12
  |
  |   /-13
  |-14
  |   \16/-15
  |
  |--17
  |
   \-18
[
            /7 /-27
         /3|
        |   \22/-9
        |
        |      /-4
        |     |
        |-15/19--6
        |     |
   /5 /18      \-10
  |     |
  |     |--16
  |     |
  |     |      /-8
  |     |-28/24
  |     |      \-23
  |     |
  |      \-25
-1|
  |--12
  |
  |--13
  |
  |--17
  |
  |-20/-26
  |
   \29/21/2 

In [108]:
def fetch_control_pos_tags(corpus, n_tags=17):
    pos_tags = {}
    for tokenlist in corpus:
        for token in tokenlist:
            pos_tags[token] = pos_tags.get(token, default=np.random.randint(1, high=n_tags+1))